##Getting yolo Ready
You do not have to do anything here

In [ ]:
!git clone https://github.com/pjreddie/darknet.git
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/g' Makefile
!sed -i 's/GPU=0/GPU=1/g' Makefile
!make

Cloning into 'darknet'...
remote: Enumerating objects: 5913, done.
remote: Total 5913 (delta 0), reused 0 (delta 0), pack-reused 5913
Receiving objects: 100% (5913/5913), 6.34 MiB | 31.98 MiB/s, done.
Resolving deltas: 100% (3919/3919), done.
/content/darknet
mkdir -p obj
mkdir -p backup
mkdir -p results
gcc -Iinclude/ -Isrc/ -DOPENCV `pkg-config --cflags opencv`  -DGPU -I/usr/local/cuda/include/ -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofast -DOPENCV -DGPU -c ./src/gemm.c -o obj/gemm.o
./src/gemm.c: In function ‘time_gpu’:
./src/gemm.c:232:9: warning: ‘cudaThreadSynchronize’ is deprecated [-Wdeprecated-declarations]
         cudaThreadSynchronize();
         ^~~~~~~~~~~~~~~~~~~~~
In file included from /usr/local/cuda/include/cuda_runtime.h:96:0,
                 from include/darknet.h:11,
                 from ./src/utils.h:5,
                 from ./src/gemm.c:2:
/usr/local/cuda/include/cuda_runtime_api.h:957:57: note: declared here
 extern __CUDA_DEPRECATE

In [ ]:
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2020-11-23 06:48:25--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M   175KB/s    in 13m 15s 

2020-11-23 07:01:41 (200 KB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



In [ ]:
!wget https://pjreddie.com/media/files/yolov3.weights

--2020-11-23 07:01:41--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights’

yolov3.weights      100%[===================>] 236.52M   104KB/s    in 26m 52s 

2020-11-23 07:28:34 (150 KB/s) - ‘yolov3.weights’ saved [248007048/248007048]



In [ ]:
!unzip /content/data.zip -d /content/darknet/
#!rm -rf '/content/darknet/labels'
#!rm -rf '/content/darknet/images'

Archive:  /content/data.zip
  inflating: /content/darknet/images/image116.jpg  
  inflating: /content/darknet/images/image117.jpg  
  inflating: /content/darknet/images/image118.jpg  
  inflating: /content/darknet/images/image119.jpg  
  inflating: /content/darknet/images/image120.jpg  
  inflating: /content/darknet/images/image121.jpg  
  inflating: /content/darknet/images/image122.jpg  
  inflating: /content/darknet/images/image123.jpg  
  inflating: /content/darknet/images/image124.jpg  
  inflating: /content/darknet/images/image125.jpg  
  inflating: /content/darknet/images/image126.jpg  
  inflating: /content/darknet/images/image127.jpg  
  inflating: /content/darknet/images/image128.jpg  
  inflating: /content/darknet/images/image129.jpg  
  inflating: /content/darknet/images/image130.jpg  
  inflating: /content/darknet/images/image131.jpg  
  inflating: /content/darknet/images/image132.jpg  
  inflating: /content/darknet/images/image133.jpg  
  inflating: /content/darknet/images

###Preparing The Config Files

In [ ]:
%%writefile cfg/mask-yolov3.cfg
[net]
# Testing
batch=10
subdivisions=2
# Training
# batch=64
# subdivisions=2
width=416
height=416
channels=3
momentum=0.9
decay=0.0005
angle=0
saturation = 1.5
exposure = 1.5
hue=.1

learning_rate=0.001
burn_in=1000
max_batches = 500200
policy=steps
steps=400000,450000
scales=.1,.1

[convolutional]
batch_normalize=1
filters=16
size=3
stride=1
pad=1
activation=leaky

[maxpool]
size=2
stride=2

[convolutional]
batch_normalize=1
filters=32
size=3
stride=1
pad=1
activation=leaky

[maxpool]
size=2
stride=2

[convolutional]
batch_normalize=1
filters=64
size=3
stride=1
pad=1
activation=leaky

[maxpool]
size=2
stride=2

[convolutional]
batch_normalize=1
filters=128
size=3
stride=1
pad=1
activation=leaky

[maxpool]
size=2
stride=2

[convolutional]
batch_normalize=1
filters=256
size=3
stride=1
pad=1
activation=leaky

[maxpool]
size=2
stride=2

[convolutional]
batch_normalize=1
filters=512
size=3
stride=1
pad=1
activation=leaky

[maxpool]
size=2
stride=1

[convolutional]
batch_normalize=1
filters=1024
size=3
stride=1
pad=1
activation=leaky

###########

[convolutional]
batch_normalize=1
filters=256
size=1
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=512
size=3
stride=1
pad=1
activation=leaky

[convolutional]
size=1
stride=1
pad=1
filters=42
activation=linear



[yolo]
mask = 3,4,5
anchors = 10,14,  23,27,  37,58,  81,82,  135,169,  344,319
classes=9
num=6
jitter=.3
ignore_thresh = .7
truth_thresh = 1
random=1

[route]
layers = -4

[convolutional]
batch_normalize=1
filters=128
size=1
stride=1
pad=1
activation=leaky

[upsample]
stride=2

[route]
layers = -1, 8

[convolutional]
batch_normalize=1
filters=256
size=3
stride=1
pad=1
activation=leaky

[convolutional]
size=1
stride=1
pad=1
filters=42
activation=linear

[yolo]
mask = 0,1,2
anchors = 10,14,  23,27,  37,58,  81,82,  135,169,  344,319
classes=9
num=6
jitter=.3
ignore_thresh = .7
truth_thresh = 1
random=1

Writing cfg/mask-yolov3.cfg


In [ ]:
%%writefile cfg/mask-obj.names
perfume
paintbrush
tissue paper box
book
playing cards
pen
butterfly knife]
toy car
slippers

Writing cfg/mask-obj.names


In [ ]:
%%writefile cfg/mask-obj.data
classes= 9
train  = train.txt  
valid  = test.txt  
names = cfg/mask-obj.names  

backup = backup/

Writing cfg/mask-obj.data


In [ ]:
!./darknet detector train cfg/mask-obj.data cfg/mask-yolov3.cfg darknet53.conv.74

Streaming output truncated to the last 5000 lines.
Loaded: 0.000052 seconds
Region 16 Avg IOU: 0.822591, Class: 0.998862, Obj: 0.975849, No Obj: 0.010764, .5R: 1.000000, .75R: 0.900000,  count: 90
Region 23 Avg IOU: 0.779312, Class: 0.996484, Obj: 0.738304, No Obj: 0.001077, .5R: 1.000000, .75R: 0.583333,  count: 36
Region 16 Avg IOU: 0.803679, Class: 0.999343, Obj: 0.952742, No Obj: 0.009303, .5R: 1.000000, .75R: 0.730769,  count: 78
Region 23 Avg IOU: 0.801021, Class: 0.996416, Obj: 0.786935, No Obj: 0.001266, .5R: 1.000000, .75R: 0.823529,  count: 51
3026: 1.012881, 1.099710 avg, 0.001000 rate, 0.204433 seconds, 30260 images
Loaded: 0.000062 seconds
Region 16 Avg IOU: 0.824998, Class: 0.998607, Obj: 0.975784, No Obj: 0.011245, .5R: 1.000000, .75R: 0.931035,  count: 87
Region 23 Avg IOU: 0.775939, Class: 0.994793, Obj: 0.753584, No Obj: 0.001195, .5R: 1.000000, .75R: 0.600000,  count: 45
Region 16 Avg IOU: 0.817678, Class: 0.999278, Obj: 0.976389, No Obj: 0.009704, .5R: 1.000000, .75

###Testing Few Images

In [ ]:
import cv2
#!./darknet detector demo cfg/mask-obj.data  cfg/mask-yolov3.cfg backup/mask-yolov3_900.weights video4.mp4v -dont_show
#!./darknet detector test cfg/mask-obj.data cfg/mask-yolov3.cfg backup/mask-yolov3_900.weights -dont_show < test.txt > result.txt

layer     filters    size              input                output
    0 conv     16  3 x 3 / 1   416 x 416 x   3   ->   416 x 416 x  16  0.150 BFLOPs
    1 max          2 x 2 / 2   416 x 416 x  16   ->   208 x 208 x  16
    2 conv     32  3 x 3 / 1   208 x 208 x  16   ->   208 x 208 x  32  0.399 BFLOPs
    3 max          2 x 2 / 2   208 x 208 x  32   ->   104 x 104 x  32
    4 conv     64  3 x 3 / 1   104 x 104 x  32   ->   104 x 104 x  64  0.399 BFLOPs
    5 max          2 x 2 / 2   104 x 104 x  64   ->    52 x  52 x  64
    6 conv    128  3 x 3 / 1    52 x  52 x  64   ->    52 x  52 x 128  0.399 BFLOPs
    7 max          2 x 2 / 2    52 x  52 x 128   ->    26 x  26 x 128
    8 conv    256  3 x 3 / 1    26 x  26 x 128   ->    26 x  26 x 256  0.399 BFLOPs
    9 max          2 x 2 / 2    26 x  26 x 256   ->    13 x  13 x 256
   10 conv    512  3 x 3 / 1    13 x  13 x 256   ->    13 x  13 x 512  0.399 BFLOPs
   11 max          2 x 2 / 1    13 x  13 x 512   ->    13 x  13 x 512
   12 con